# Views & Export Workflows

This notebook demonstrates **state management and data export** capabilities.

## Features Demonstrated

- 💾 **Named Views** - Save filter/sort/label combinations
- 📤 **CSV Export** - Export filtered data subsets
- 📋 **View Export** - Save view configurations as JSON
- ⚙️ **Config Export** - Export display configuration
- 🔄 **View Restoration** - Load saved views across sessions
- 🎯 **Workflow Patterns** - Common analysis workflows

## Use Case: Product Performance Analysis

Scenario: Analyze sales performance across:
- 50 products
- 4 categories (Electronics, Apparel, Food, Home)
- 3 regions (North, South, East)
- Multiple performance tiers

We'll create several **pre-defined views** for common analysis scenarios.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pathlib import Path
import sys
from datetime import datetime, timedelta

sys.path.insert(0, str(Path.cwd().parent.parent))

from trelliscope import Display
from trelliscope.dash_viewer import create_dash_app

np.random.seed(42)

## Generate Product Sales Data

In [ ]:
n_products = 50
categories = ['Electronics', 'Apparel', 'Food', 'Home']
regions = ['North', 'South', 'East']
n_days = 90

print(f"Generating sales data for {n_products} products over {n_days} days...\n")

products_data = []

for i in range(n_products):
    product_id = f'P{i+1:03d}'
    category = categories[i % len(categories)]
    region = regions[i % len(regions)]
    
    # Generate daily sales
    base_sales = np.random.uniform(50, 200)
    trend = np.random.uniform(-0.5, 1.0)
    seasonality = np.random.uniform(10, 30) * np.sin(np.linspace(0, 4*np.pi, n_days))
    noise = np.random.randn(n_days) * 10
    
    daily_sales = base_sales + np.linspace(0, trend * n_days, n_days) + seasonality + noise
    daily_sales = np.maximum(daily_sales, 0)  # No negative sales
    
    dates = [datetime(2024, 1, 1) + timedelta(days=d) for d in range(n_days)]
    
    # Calculate metrics
    total_revenue = daily_sales.sum()
    avg_daily = daily_sales.mean()
    growth_rate = (daily_sales[-7:].mean() - daily_sales[:7].mean()) / daily_sales[:7].mean() * 100
    
    # Determine performance tier
    if total_revenue > 12000:
        tier = 'Top'
    elif total_revenue > 8000:
        tier = 'High'
    elif total_revenue > 5000:
        tier = 'Medium'
    else:
        tier = 'Low'
    
    # Create Plotly chart
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=dates,
        y=daily_sales,
        mode='lines',
        name='Daily Sales',
        line=dict(color='#2c7fb8', width=2),
        fill='tozeroy',
        fillcolor='rgba(44, 127, 184, 0.2)',
        hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Sales: $%{y:.0f}<extra></extra>'
    ))
    
    # Add 7-day moving average
    ma7 = pd.Series(daily_sales).rolling(window=7).mean()
    fig.add_trace(go.Scatter(
        x=dates,
        y=ma7,
        mode='lines',
        name='7-day MA',
        line=dict(color='#d95f02', width=2, dash='dash'),
        hovertemplate='<b>%{x|%Y-%m-%d}</b><br>MA7: $%{y:.0f}<extra></extra>'
    ))
    
    fig.update_layout(
        title=f"{product_id} - {category} ({region})",
        xaxis_title='Date',
        yaxis_title='Daily Sales ($)',
        hovermode='x unified',
        height=400,
        showlegend=True,
        legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    fig.update_xaxes(showgrid=True, gridcolor='#e0e0e0')
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0')
    
    products_data.append({
        'product_id': product_id,
        'category': category,
        'region': region,
        'total_revenue': total_revenue,
        'avg_daily_sales': avg_daily,
        'growth_rate': growth_rate,
        'performance_tier': tier,
        'panel': fig
    })
    
    if (i + 1) % 10 == 0:
        print(f"  Generated {i+1}/{n_products} products...")

df = pd.DataFrame(products_data)
print(f"\n✓ Generated {len(df)} product panels")

In [ ]:
# Summary statistics
print("Dataset Summary:")
print(f"\nBy Category:")
print(df.groupby('category')['total_revenue'].agg(['count', 'mean', 'sum']).round(0))

print(f"\nBy Performance Tier:")
print(df.groupby('performance_tier')['total_revenue'].agg(['count', 'mean']).round(0))

print(f"\nBy Region:")
print(df.groupby('region')['total_revenue'].agg(['count', 'mean']).round(0))

## Create Display with Pre-Configured Views

We'll define several common analysis views that users can quickly switch between.

In [ ]:
display = (
    Display(
        df,
        name="product_performance_views",
        description="Product Performance Analysis with Pre-configured Views and Export"
    )
    .set_panel_column("panel")
    .infer_metas()
    .set_default_layout(ncol=3, nrow=2)
    .set_default_labels(["product_id", "category", "performance_tier", "total_revenue"])
)

print("✓ Display configured")
print(f"  Products: {len(df)}")
print(f"  Categories: {df['category'].nunique()}")
print(f"  Regions: {df['region'].nunique()}")
print(f"  Performance Tiers: {df['performance_tier'].nunique()}")

## Launch Viewer - Views & Export Tutorial

### 🎯 Workflow 1: Create and Save Custom Views (3 minutes)

**Scenario**: You need to regularly review specific product segments.

#### View 1: "Top Performers"
```
1. Filter: performance_tier = "Top"
2. Sort: total_revenue (descending)
3. Click "Save View" button
4. Name: "Top Performers"
5. Click Save
```

#### View 2: "Electronics - High Growth"
```
1. Filter: category = "Electronics"
2. Filter: growth_rate > 0
3. Sort: growth_rate (descending)
4. Labels: Add "growth_rate" to labels
5. Save View → Name: "Electronics - High Growth"
```

#### View 3: "North Region - All Tiers"
```
1. Filter: region = "North"
2. Sort: performance_tier (ascending), total_revenue (descending)
3. Save View → Name: "North Region - All Tiers"
```

#### View 4: "Underperformers Need Attention"
```
1. Filter: performance_tier = "Low"
2. Filter: growth_rate < 0 (negative growth)
3. Sort: total_revenue (ascending)
4. Save View → Name: "Underperformers Need Attention"
```

---

### 🎯 Workflow 2: Navigate Between Views (1 minute)

```
1. Use Views dropdown in sidebar
2. Select "Top Performers" → See filtered results
3. Select "Electronics - High Growth" → Filters change automatically
4. Select "Underperformers Need Attention" → See concerning products
5. Select "Default" → Reset to original state
```

**Benefit**: Instantly switch context without re-applying filters!

---

### 🎯 Workflow 3: Export Filtered Data (2 minutes)

#### Export CSV - Top Performers
```
1. Load View: "Top Performers"
2. Scroll down in sidebar to "Export" section
3. Click "Export Data (CSV)"
4. File downloads: top_performers_data.csv
5. Open in Excel/Python for further analysis
```

**What's exported**: Only the filtered products with all metadata columns

#### Export View Configuration
```
1. Create a complex filter combination:
   - category = "Electronics" OR "Apparel"
   - performance_tier = "Top" OR "High"
   - region = "North"
2. Click "Export View (JSON)"
3. Save as: my_custom_view.json
4. Share with colleagues or load later
```

#### Export Display Config
```
1. Click "Export Config"
2. Downloads: display_config.json
3. Contains: All display metadata, cognostics definitions
4. Use for: Documentation, reproducibility
```

---

### 🎯 Workflow 4: Iterative Analysis Pattern (5 minutes)

**Real-world scenario**: Find products needing intervention

```
Step 1: Overview
  - Start with all products (Default view)
  - Sort by total_revenue descending
  - Identify general trends

Step 2: Deep Dive - Category Performance
  - Filter: category = "Electronics"
  - Sort: growth_rate descending
  - Look for negative growth → Click panels to inspect
  - Save View: "Electronics Deep Dive"
  - Export CSV for detailed analysis

Step 3: Regional Comparison
  - Clear filters (Ctrl+R)
  - Filter: performance_tier = "Low"
  - Group by region mentally
  - Identify if problems are regional
  - Save View: "Low Performers by Region"

Step 4: Action Items
  - Combine filters:
    * performance_tier = "Low"
    * growth_rate < -5 (declining > 5%)
  - Sort: total_revenue descending (prioritize by impact)
  - Export CSV → Share with product managers
  - Save View: "Priority Intervention List"

Step 5: Track Progress
  - Next week: Load "Priority Intervention List" view
  - Check if metrics improved
  - Update view or create new one
```

---

### 💡 Pro Tips

**View Management:**
- Name views descriptively: "Q4 Top Sellers" not "View 1"
- Create views for recurring reports
- Delete outdated views to keep list clean
- Share view JSON files with team members

**Export Strategies:**
- **CSV exports** preserve current filter state
- **View exports** save filter/sort/label configuration
- **Config exports** document display structure
- Use exports for reproducible analysis pipelines

**Keyboard Shortcuts:**
- `Ctrl+S` - Quick save current view
- `Ctrl+R` - Reset all filters (start fresh)
- `/` then type category name - Quick filter

---

The viewer opens at http://localhost:8057

**Follow the workflows above to master views and exports!**

In [ ]:
app = create_dash_app(display)
app.run(debug=False, host='127.0.0.1', port=8057)